## 반복 실험을 위한 Sacred
- [Sacred Github](https://github.com/IDSIA/sacred)
- Sacred란?
    - Sacred is a tool to help you configure, organize, log and reproduce experiments developed at IDSIA
    - 머신러닝 모델링을 진행할 때 설정을 저장해주고 관리하는 것을 도와주는 도구
- 왜 필요한가요?
    - Kaggle에서 자주 발생하는 예시
        - 사용한 Feature는?
        - 사용한 파라미터는?
        - 그 결과는?
    - 다양한 실험을 빠르게 진행하며, 손으로 기록하지 않고 자동으로 기록될 수 있도록 도와줄 도구가 필요
- Scratch로 구현하면?
    - Logger를 정의해서 필요할 때마다 Logger에 Feature, 파라미터 등을 저장
    - 값을 보기 위해 Logger를 Parsing
- Sacred는 위 방법을 데코레이터로 쉽게 사용할 수 있도록 도와줌
    - MLOps와 관련되는 부분으로, 국내의 다양한 머신러닝 강의에서 이런 부분을 다루지 않아 추가!

- Sacred의 Main mechanisms
    - ConfigScopes : 함수의 local 변수를 편리하게 다룰 수 있음 @ex.config 데코레이터로 사용
    - Config Injection : 모든 함수에 있는 설정을 접근할 수 있음
    - Command-line interface : 커맨드 라인으로 파라미터를 바꿔서 실행할 수 있음
    - Observers : 실험의 모든 정보를 Observers에게 제공해 저장. MongoDB / S3 등 => 이번 강의에선 그냥 로컬에 저장
    - Automatic seeding : 실험의 무작위를 컨트롤할 때 도와줌

In [1]:
!pip3 install sacred

  Created wheel for sacred: filename=sacred-0.8.1-py2.py3-none-any.whl size=105022 sha256=5908309a1d159d1ad253c7cb19c64da7d73bc8cb814a04c822132b4c66dec534
  Stored in directory: c:\users\영현\appdata\local\pip\cache\wheels\3c\ac\e1\2f746c47edc95a1cf43119706c787efd9c307a8b3d4a649308
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=e2940faf40925ad73867186882f4864e68bcb6976c717592a2ebb332798d84b6
  Stored in directory: c:\users\영현\appdata\local\pip\cache\wheels\72\b0\3f\1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-py3-none-any.whl size=20074 sha256=62407f3a5dc7ef2ebd506f1273f9fd414bfca48b331390f491682e1538783e54
  Stored in directory: c:\users\영현\appdata\local\pip\cache\wheels\d7\59\0d\58c5e576d9192261fa3da00466eebe6f7a1ac1873a7ab1f2ce
Successfully built sacred docopt py-cpuinfo


You should consider upgrading via the 'c:\users\hyun\python.exe -m pip install --upgrade pip' command.


### 공식 홈페이지 Sacred 예시

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from ipywidgets import interact
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import os
from numpy.random import permutation
from sklearn import svm, datasets
from sacred import Experiment
from sacred.observers import FileStorageObserver

plt.style.use('ggplot')
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

PROJECT_ID='nyc-taxi-demand-predict' # 여기에 여러분들의 프로젝트 ID를 넣어주세요

In [2]:
# Notebook에선 interactive=True 지정해줘야 하고, 스크립트 형식이라면 이 옵션이 필요 없음
ex = Experiment('iris_rbf_svm', interactive=True)
# Experiment: 실험 객체를 만들어줘야해

# Decorator로 config 저장
@ex.config
def cfg():
    C = 1.0
    gamma = 0.7

# ex.main을 지정 => 이 때 cfg에 있는 인자들이 자동으로 injection됨
# 또한 Notebook에선 ex.main을 쓰고 스크립트 파일에선 ex.automain을 사용
@ex.main
def run(C, gamma):
    iris = datasets.load_iris()
    per = permutation(iris.target.size)
    iris.data = iris.data[per]
    iris.target = iris.target[per]
    clf = svm.SVC(C, 'rbf', gamma=gamma)
    clf.fit(iris.data[:90],
          iris.target[:90])
    return clf.score(iris.data[90:],
                   iris.target[90:])

In [3]:
run_result = ex.run()

WARNING - iris_rbf_svm - No observers have been added to this run
INFO - iris_rbf_svm - Running command 'run'
INFO - iris_rbf_svm - Started
INFO - iris_rbf_svm - Result: 0.9833333333333333
INFO - iris_rbf_svm - Completed after 0:00:00


In [4]:
run_result.config

{'C': 1.0, 'gamma': 0.7, 'seed': 969061621}

In [5]:
run_result.result
# score return

0.9833333333333333

In [7]:
run_result.experiment_info

{'name': 'iris_rbf_svm',
 'base_dir': 'C:\\Users\\영현\\STUDY\\Fast Campus\\머신러닝과 데이터분석 A-Z\\8. 실전프로젝트\\3. 뉴욕(NYC) 택시 수요 예측',
 'sources': [],
 'dependencies': ['ipython==7.9.0',
  'ipywidgets==7.5.1',
  'matplotlib==3.2.1',
  'numpy==1.17.4',
  'pandas==0.25.3',
  'sacred==0.8.1',
  'scikit-learn==0.22.2.post1',
  'seaborn==0.9.0'],
 'repositories': [],
 'mainfile': None}

### 기존 프로젝트에 통합

In [8]:
ex = Experiment('nyc-demand-prediction', interactive=True)

# experiment_dir가 없으면 폴더 생성하고 FileStorageObserver로 저장
experiment_dir = os.path.join('./', 'experiments')
if not os.path.isdir(experiment_dir): 
    os.makedirs(experiment_dir)
ex.observers.append(FileStorageObserver.create(experiment_dir))

### 전처리

In [9]:
%%time
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id=PROJECT_ID)

INFO - google.cloud.bigquery.opentelemetry_tracing - This service is instrumented using OpenTelemetry. OpenTelemetry could not be imported; please add opentelemetry-api and opentelemetry-instrumentation packages in order to get BigQuery Tracing data.
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable onattempt 1 of 3. Reason: timed out
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable onattempt 2 of 3. Reason: timed out
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable onattempt 3 of 3. Reason: timed out
WARNING - google.auth._default - Authentication failed using Compute Engine authentication due to unavailable metadata server.
Downloading: 100%|██████████████████████████████████████████████████████████| 87020/87020 [00:07<00:00, 10886.17rows/s]
INFO - pandas_gbq.gbq - Total time taken 10.67 s.
Finished at 2020-12-03 14:55:03.


Wall time: 20.9 s


In [10]:
base_df.head(3)
# cnt: target feature

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt
0,10037,2015-01-01,1,1,3,0,0,26
1,10170,2015-01-01,1,1,3,0,0,44
2,11238,2015-01-01,1,1,3,0,0,95


### Feautre Engineering

In [11]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base_df[['zip_code']])
ohe_output = enc.transform(base_df[['zip_code']]).toarray()
ohe_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+enc.categories_[0])], axis=1)
ohe_df['log_cnt'] = np.log10(ohe_df['cnt'])

In [12]:
def split_train_and_test(df, date):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    
    df : 시계열 데이터 프레임
    date : 기준점 날짜
    """
    train_df = df[df['pickup_hour'] < date]
    test_df = df[df['pickup_hour'] >= date]
    return train_df, test_df

### Train / Test 나누기

In [13]:
train_df, test_df = split_train_and_test(ohe_df, '2015-01-24')

In [14]:
train_df.tail()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801,log_cnt
65113,11415,2015-01-23 23:00:00,1,23,4,23,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
65114,10040,2015-01-23 23:00:00,1,23,4,23,0,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30103
65115,10271,2015-01-23 23:00:00,1,23,4,23,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
65116,11204,2015-01-23 23:00:00,1,23,4,23,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
65117,11356,2015-01-23 23:00:00,1,23,4,23,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


- 사용하지 않을 컬럼 삭제

In [15]:
del train_df['zip_code']
del train_df['pickup_hour']
del test_df['zip_code']
del test_df['pickup_hour']

In [16]:
train_df.head(2)

,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,zip_code_10003,zip_code_10004,...,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801,log_cnt
0,1,1,3,0,0,26,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.414973
1,1,1,3,0,0,44,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.643453


In [17]:
y_train_raw = train_df.pop('cnt')
y_train_log = train_df.pop('log_cnt')
y_test_raw = test_df.pop('cnt')
y_test_log = test_df.pop('log_cnt')

In [18]:
y_true = y_test_raw.values.copy()

In [19]:
x_train = train_df.copy()
x_test = test_df.copy()

In [20]:
def evaluation(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    score = pd.DataFrame([mape, mae, mse], index=['mape', 'mae', 'mse'], columns=['score']).T
    return score

### 실험 설정
- 위에서 ex = Experiment('nyc-demand-prediction', interactive=True)했는데, ex.config로 설정을 저장
- ex.capture는 해당 설정을 사용해 함수를 리턴
- ex.main은 실험이 실행될 때 진행할 내용을 담음

In [31]:
@ex.config
def config():
    fit_intercept=True
    normalize=False

In [32]:
@ex.capture
def get_model(fit_intercept, normalize):
    return LinearRegression(fit_intercept, normalize)

# capture: 함수를 만들어서 기입을 할 수 있는 decorate

In [33]:
# _log과 _run은 별도로 정의하지 않아도 함수의 인자로 사용 가능
@ex.main
def run(_log, _run):
    lr_reg = get_model() # 인자 안넣어도 돼 (config에서 설정해줬기때문)
    lr_reg.fit(x_train, y_train_raw)
    pred = lr_reg.predict(x_test)
    # log File에 로그 저장
    _log.info("Predict End")
    score = evaluation(y_test_raw, pred)
    _run.log_scalar('model_name', lr_reg.__class__.__name__)
    
    # Metrics쪽에 저장하고 싶으면 아래처럼 사용 (추천)
    _run.log_scalar('metrics', score)
    
    # Result쪽에 저장하고 싶으면 아래처럼 사용
    return score.to_dict()

In [34]:
experiment_result = ex.run()

INFO - nyc-demand-prediction - Running command 'run'
INFO - nyc-demand-prediction - Started run with ID "2"
INFO - run - Predict End
INFO - nyc-demand-prediction - Result: {'mape': {'score': 225017760230.73145}, 'mae': {'score': 2300181639.6865945}, 'mse': {'score': 5.038255261890111e+21}}
INFO - nyc-demand-prediction - Completed after 0:00:04


In [25]:
experiment_result.config

{'fit_intercept': True, 'normalize': False, 'seed': 696760466}

### Experiment 확인하기 위한 Parser
- Experiment에서 log찍는 방식에 따라 사용할 함수가 다름
    - 1) \_run.log\_scalar에 metrics을 저장하는 경우 : 추천
    - 2) @ex.main의 함수에 결과를 return하는 경우 
        - 위 결과는 1,2번 둘 다 되어있는 경우

In [70]:
# 1) _run.log_scalar에 metrics을 저장하는 경우
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/metrics.json') as json_file:
        json_data = json.load(json_file)
    with open(f'./experiments/{ex_id}/config.json') as config_file:
        config_data = json.load(config_file)
    with open(f'./experiments/{ex_id}/run.json') as run_file:
        run_data = json.load(run_file)
    
    output_df = pd.DataFrame(json_data['model_name']['values'], columns=['model_name'], index=['score'])
    output_df['experiment_num'] = ex_id
    output_df['config'] = str(config_data)
    run_df = pd.DataFrame(run_data['result'])
    output_df = pd.concat([output_df, run_df], axis=1)
    output_df = output_df.round(2)
    return output_df

In [59]:
# 2) @ex.main의 함수에 결과를 return하는 경우
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/run.json') as json_file:
        json_data = json.load(json_file)
    output = pd.DataFrame(json_data['result'])
    return output

In [71]:
parsing_output(2)

,model_name,experiment_num,config,mae,mape,mse
score,LinearRegression,2,"{'fit_intercept': True, 'normalize': False, 's...",2.300182e+09,2.250178e+11,5.038255e+21


### 더 자세한 내용이 궁금하면
- [Sacred Github](https://github.com/IDSIA/sacred)
- [머신러닝 실험을 도와줄 Python Sacred 소개](https://zzsza.github.io/mlops/2019/07/21/python-sacred/)
- [Sacred와 Omniboard를 활용한 실험 및 로그 모니터링](https://zzsza.github.io/mlops/2019/07/22/sacred-with-omniboard/)